# 1. 데이터 불러오기

In [1]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings(action='ignore')

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [2]:
df = pd.read_csv('C:/Users/w7e7r/Desktop/스파르타/심화프로젝트/train.csv')

# 2. 결측치, 이상치

## 결측치

### 주택소유상태

In [3]:
df = df.drop(index=df[df['주택소유상태'] == 'ANY'].index)

In [4]:
df['대출기간'] = df['대출기간'].str.strip()
df['대출기간'] = df['대출기간'].str.split(' ', expand=True)[0]
df['대출기간'] = df['대출기간'].astype(int)

In [5]:
df['대출기간'].unique()

array([36, 60])

### 근로기간

In [6]:
df_un = df[df['근로기간'] == 'Unknown']
df_n = df[df['근로기간'] != 'Unknown']

In [7]:
def year_encoder(value):
    if value == 'Unknown':
        return 0
    else:
        # 숫자만 추출
        numeric_value = re.findall('\d+', value)
        if numeric_value:
            return int(numeric_value[0])
        else:
            return 0

# 근로기간 데이터 변환
df_n['근로기간'] = df_n['근로기간'].apply(year_encoder)
df_un['근로기간'] = df_un['근로기간'].apply(year_encoder)

In [8]:
work_year = list(df_n['근로기간'].unique())
work_year = sorted(work_year)
work_year

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [9]:
numeric_col = ['대출기간', '대출금액', '연간소득', '부채_대비_소득_비율', '총계좌수', '최근_2년간_연체_횟수', '총상환원금', '총상환이자', '총연체금액', '연체계좌수', '근로기간']
category_col = [ '주택소유상태', '대출목적']

In [10]:
un_income = df_un['연간소득'].mean()
un_debt_ratio = df_un['부채_대비_소득_비율'].mean()

un_feature = [un_income, un_debt_ratio]

In [11]:
un_feature = ['연간소득', '부채_대비_소득_비율']

sses = []

for year in work_year:
    sse_list = []
    
    temp = df_n[df_n['근로기간'] == year]
    
    for col in un_feature:
        feature = temp[col]
        data = (feature.value_counts() / feature.value_counts().values.sum()).sort_index()
        
        error = data - df_un[col].mean()  
        
        se = error ** 2
        
        sse_list.append(se.values.sum())
    
    print(f'{year}년 SSE : {sum(sse_list)}')


1년 SSE : 5.058911843133164e+18
2년 SSE : 3.9800660898169257e+18
3년 SSE : 3.509893447324445e+18
4년 SSE : 3.076168296497903e+18
5년 SSE : 2.8975755873341117e+18
6년 SSE : 2.452916189007704e+18
7년 SSE : 2.4747846840073313e+18
8년 SSE : 2.6315088981716347e+18
9년 SSE : 2.401889700675149e+18
10년 SSE : 1.1998514256355176e+19


- 5년

In [12]:
print('unknown average income:', un_income)
print('unknwon debt ratio:', un_debt_ratio)

unknown average income: 60371758.02151296
unknwon debt ratio: 22.716878857344383


In [13]:
for year in work_year:
    temp = df_n[df_n['근로기간'] == year]
    
    error_income = temp['연간소득'].mean() 
    error_debt_ratio = temp['부채_대비_소득_비율'].mean()
    
    print(f'{year} year error in average income: {error_income}')
    print(f'{year} year error in debt to income ratio: {error_debt_ratio}')

1 year error in average income: 87926613.47774932
1 year error in debt to income ratio: 18.6887161741297
2 year error in average income: 90848452.21349113
2 year error in debt to income ratio: 18.61789822485207
3 year error in average income: 92266220.33585398
3 year error in debt to income ratio: 18.83573272490222
4 year error in average income: 92732040.60987832
4 year error in debt to income ratio: 18.773478883321403
5 year error in average income: 94352701.37687555
5 year error in debt to income ratio: 18.94184112974404
6 year error in average income: 93841795.36912751
6 year error in debt to income ratio: 19.345937016004132
7 year error in average income: 95928259.36234924
7 year error in debt to income ratio: 19.470608285264813
8 year error in average income: 95570175.70703764
8 year error in debt to income ratio: 19.43097790507365
9 year error in average income: 96307501.69551282
9 year error in debt to income ratio: 19.617873931623933
10 year error in average income: 103032164.

In [14]:
for year in work_year:
    temp = df_n[df_n['근로기간'] == year]
    
    error_income = temp['연간소득'].mean() - un_income
    error_debt_ratio = temp['부채_대비_소득_비율'].mean() - un_debt_ratio
    
    print(f'{year} year error in average income: {error_income}')
    print(f'{year} year error in debt to income ratio: {error_debt_ratio}')

1 year error in average income: 27554855.456236355
1 year error in debt to income ratio: -4.028162683214685
2 year error in average income: 30476694.191978164
2 year error in debt to income ratio: -4.098980632492314
3 year error in average income: 31894462.314341016
3 year error in debt to income ratio: -3.881146132442165
4 year error in average income: 32360282.588365354
4 year error in debt to income ratio: -3.943399974022981
5 year error in average income: 33980943.35536259
5 year error in debt to income ratio: -3.775037727600342
6 year error in average income: 33470037.34761455
6 year error in debt to income ratio: -3.370941841340251
7 year error in average income: 35556501.34083628
7 year error in debt to income ratio: -3.2462705720795704
8 year error in average income: 35198417.68552468
8 year error in debt to income ratio: -3.285900952270733
9 year error in average income: 35935743.67399985
9 year error in debt to income ratio: -3.0990049257204504
10 year error in average income

- 7년

In [15]:
for year in work_year:
    temp = df_n[df_n['근로기간'] == year]
    
    error_income = temp['연간소득'].mean() - un_income
    income_ratio = un_income / error_income 
    
    error_debt_ratio = temp['부채_대비_소득_비율'].mean() - un_debt_ratio
    debt_ratio = un_debt_ratio / error_debt_ratio
    
    print(f'{year}년 평균 소득 비율: {income_ratio:.2f}')
    print(f'{year}년 평균 부채 대비 소득 비율 비율: {debt_ratio:.2f}')

1년 평균 소득 비율: 2.19
1년 평균 부채 대비 소득 비율 비율: -5.64
2년 평균 소득 비율: 1.98
2년 평균 부채 대비 소득 비율 비율: -5.54
3년 평균 소득 비율: 1.89
3년 평균 부채 대비 소득 비율 비율: -5.85
4년 평균 소득 비율: 1.87
4년 평균 부채 대비 소득 비율 비율: -5.76
5년 평균 소득 비율: 1.78
5년 평균 부채 대비 소득 비율 비율: -6.02
6년 평균 소득 비율: 1.80
6년 평균 부채 대비 소득 비율 비율: -6.74
7년 평균 소득 비율: 1.70
7년 평균 부채 대비 소득 비율 비율: -7.00
8년 평균 소득 비율: 1.72
8년 평균 부채 대비 소득 비율 비율: -6.91
9년 평균 소득 비율: 1.68
9년 평균 부채 대비 소득 비율 비율: -7.33
10년 평균 소득 비율: 1.42
10년 평균 부채 대비 소득 비율 비율: -7.22


In [16]:
df_simple = pd.concat([df_n, df_un], axis=0).reset_index(drop=True)